In [45]:
from dash import Dash, dash_table as dt, dcc, html
from dash.dependencies import Input, Output, State
import pandas as pd 
from Librerias_SAP import SAP_GUI, Funtions
import re
from collections import OrderedDict
import dash
import numpy as np
import dash_bootstrap_components as dbc
from datetime import datetime, timedelta
import plotly.graph_objects as go
from datetime import datetime, timedelta

In [46]:
# Disposiciones ZSD110
Name_ZSD110_FIRME="Pendiente_Firme"  # Revisar suele no guardar
Name_ZSD110_MERCADEO="Pendiente_Mercadeo"  # Revisar suele no guardar
Name_ZSD110_TOTAL="Pendiente_Total"  # Revisar suele no guardar

default_column_Canal01_Pendientes=['Can.distr.', 'Denomin.', 'Seccion', 'Estado', 'Solic.', 'Nombre',
       'Pedido', 'Clase Pedi', 'Creado el', 'Pedido.1', 'Posición', 'Material',
       'Vlr.Neto P', 'Moneda', 'Cantidad P', 'Cantidad C', 'UM venta',
       'Vlr.Neto C', 'Status Glo', 'Status Tot', 'Status Ent', 'Status Ent.1',
       'Status Ent.2']  

default_column_Canal01_Entregado=['Can.distr.', 'Denomin.', 'Seccion', 'Estado', 'Solic.', 'Nombre',
       'Entrega', 'Posicion E', 'F.Creacion', 'Pedido', 'Clase Pedi',
       'F.Creacion.1', 'Pedido.1', 'Posicion P', 'Material', 'Moneda',
       'Cantidad E', 'UM venta', 'Valor Neto', 'StTotPick.', 'StatGlWM',
       'StTotMovMe', 'Stat.fact.', 'Status PT']

default_column_Pedidos=['OrgVt', 'CDis', 'BqEn', 'Solic.', 'PrimFeEntr', 'ÚltEntrega',
       'FePrefEnt.', 'ClVt', 'Valor neto', 'Mon.', 'Doc.venta', 'ST',
       'Solicitante', 'Nº pedido cliente', 'Descrip.breve', 'Nº de cliente 1',
       'Contador de pedidos', 'Dirección', 'Barrio', 'Población', 'Teléfono']

Columns_Data_ZSD10_Exito_Cencosub_ZVMI=['Fecha doc.', 'Solicitante', 'Cliente', 'Nº de cliente 1', 'Marca',
       'Material', 'Número de material', 'Unidades', 'UM', 'ValorNeto',
       'Nº pedido cliente', 'Doc.venta', 'ClVt', 'Solic.', 'OrgVt', 'CDis',
       'OfVta', 'GVen', 'Organización de ventas', 'Mon.', 'Jquía.productos',
       'Total Pedidos', 'Plataforma', 'Fe.PedCpas', 'Creado por']

Columns_Data_ZSD038_Exito_Cencosub_Rechazados=['Doc.comer.', 'Fecha doc.', 'Destinat.', 'Nombre 1', 'Cód.Vend.',
       'Nombre vendedor', 'MR', 'Denominación', 'Material', 'Denominación.1',
       'Marca', 'Categoría', 'Ctd.confirmada', 'Valor neto', 'Mon.', 'CDis',
       'Solic.', 'Nombre']

Columns_Data_ZSD10=['Fecha doc.', 'Solicitante', 'Cliente', 'Nº de cliente 1', 'Marca',
       'Material', 'Número de material', 'Unidades', 'UM', 'ValorNeto',
       'Nº pedido cliente', 'Doc.venta', 'ClVt', 'Solic.', 'OrgVt', 'CDis',
       'OfVta', 'GVen', 'Organización de ventas', 'Mon.', 'Jquía.productos',
       'Total Pedidos', 'Plataforma', 'Fe.PedCpas', 'Creado por']

Column_Data_ZSD76=['Solic.', 'Solicitante', 'Fecha salida mercancía', 'Material',
       'Número de material', 'Cantidad entrega', 'UM', 'MARCA', 'CATEGORIA',
       'Entrega', 'ClEnt', 'Clase de entrega', 'ClVt',
       'Clase de documento de ventas', 'OrgVt', 'CDis', 'Doc.venta',
       'Prc.neto', 'Mon.', 'Destinat.', 'Destinatario de mercancías',
       'Nº pedido cliente', 'Fecha creación Pedido', 'Cont.']

"""----------------------------------------------LEER ARCHIVOS--------------------------------"""
#Data Pendientes
Canal01_Pendiente=pd.read_csv("Archivos_CSV\\Canal01_PEN.txt",skiprows=1,delimiter="\t")
Canal01_Pendiente=Funtions.Clean_Columns(Canal01_Pendiente)
Canal01_Pendiente=Funtions.default_column(default_column_Canal01_Pendientes,Canal01_Pendiente)

#Data Entregados
Canal01_Entregado=pd.read_csv("Archivos_CSV\\Canal01_ENT.txt",skiprows=1,delimiter="\t")
Canal01_Entregado=Funtions.Clean_Columns(Canal01_Entregado)
Canal01_Entregado=Funtions.default_column(default_column_Canal01_Entregado,Canal01_Entregado)


In [47]:
"""""-------------------------------------------Limpio los datos-----------------------------------"""

# Esta session combierte los valores monetarios en enteros str a int para posteriores calculos
Canal01_Pendiente['Vlr.Neto P'], Canal01_Pendiente['Cantidad P']=Funtions.Clean_Num_List(Canal01_Pendiente['Vlr.Neto C'], Canal01_Pendiente['Cantidad P'])
Canal01_Entregado['Valor Neto'], Canal01_Entregado['Cantidad E']=Funtions.Clean_Num_List(Canal01_Entregado['Valor Neto'],Canal01_Entregado['Cantidad E'])

# Esta session combierte colunmas de float .0 a enteros int, tambien recibe valores Null
Canal01_Pendiente['Pedido.1'],Canal01_Entregado['Pedido.1']=Funtions.Estandarizo_Pedidos(Canal01_Pendiente['Pedido.1'],Canal01_Entregado['Pedido.1'])


## ¿Preguntar cantidad pendiente P y C significado

In [48]:
"""Seccion para coompletar los valores necesarios para trabajar"""

Agenda=["85","20","146","149","50","138","45"]
# En este caso solo aplica para Entrega y pendiente en el cual completa con 00 los pedidos del Exito
Canal01_Entregado['Pedido.1']=Funtions.complete_pedidos(Canal01_Entregado['Pedido.1'],Agenda)
Canal01_Pendiente['Pedido.1']=Funtions.complete_pedidos(Canal01_Pendiente['Pedido.1'],Agenda)

In [49]:
"""---------------------------------------Unifico los dataframes de pedido y entregas--------------------------------------------------------------"""
Default_Column_Informe=['Canal','Denomin.','Solic.','Nombre','Pedido','Clase Pedido','Material','Valor Neto','Cantidad']

longuitud_Pendiente=len(Canal01_Pendiente)
longuitud_Entrega=len(Canal01_Entregado)


Canal01_Pendiente[['Can.distr.','Denomin.','Solic.','Nombre','Pedido.1','Clase Pedi','Material','Vlr.Neto P','Cantidad P']]
Canal01_Entregado[['Can.distr.', 'Denomin.','Solic.','Nombre','Pedido.1','Clase Pedi','Material','Valor Neto','Cantidad E']]


Canal01_Pendiente_Informe=Canal01_Pendiente[['Can.distr.','Denomin.','Solic.','Nombre','Pedido.1','Clase Pedi','Material','Vlr.Neto P','Cantidad P']]
Canal01_Entregado_Informe=Canal01_Entregado[['Can.distr.', 'Denomin.','Solic.','Nombre','Pedido.1','Clase Pedi','Material','Valor Neto','Cantidad E']]


print(longuitud_Pendiente,longuitud_Entrega)


"""---En esta sección analizar si borro duplicados-----"""

Canal01_Entregado_Pendiente_Informe=pd.concat([Funtions.default_column(Default_Column_Informe,Canal01_Pendiente_Informe),Funtions.default_column(Default_Column_Informe,Canal01_Entregado_Informe)]).drop_duplicates().reset_index(drop=True)

3720 869


In [50]:
""""------------------------------------ Estandarizo datos de columnas del informe de las lineas que deberian ser numericos------------------------------------------------"""

try:
    Canal01_Entregado_Pendiente_Informe['Valor Neto'],Canal01_Entregado_Pendiente_Informe['Cantidad']=Funtions.Clean_Num_List(Canal01_Entregado_Pendiente_Informe['Valor Neto'],Canal01_Entregado_Pendiente_Informe['Cantidad'])
except:
    print("intento")
    pass


In [51]:
"""------------------------Optener datos de Agenda_Exito y Cencosub------------------"""
#Agenda Exito y Cencosub
Agenda_Exito=["0085","0020","0146","0149","0050","0138","0045"]
Agenda_Cencosub=["93","122","127","95","60"]


"""-----------------------------Aplico filtro en Agenda exito Entrega----------------------------------"""
Filtro_Agenda_Exito_Entrega_Pendiente=Funtions.Search_Agenda_Exito(Canal01_Entregado_Pendiente_Informe['Pedido'],Agenda_Exito)
Filtro_Agenda_Cencosub_Entrega_Pendiente=Funtions.Search_agenda_Cencosub(Canal01_Entregado_Pendiente_Informe["Pedido"],Agenda_Cencosub)
Filtro_Sin_Agenda_Cencosub_Entrega_Pendiente=Filtro_Agenda_Exito_Entrega_Pendiente+Filtro_Agenda_Cencosub_Entrega_Pendiente

In [52]:
"""-------------------------------------------------Aplico filtro para el consolidados------------------------------------------------------------"""


"""------------------------------Canal 01 Entrega con agenda Exito-----------------------"""
#Agenda Exito
Filtro_Canal01_Entregado_Pendiente_Exito=(Canal01_Entregado_Pendiente_Informe['Pedido'].isin(Filtro_Agenda_Exito_Entrega_Pendiente))   #Excluyo con ~
Canal01_Entregado_Pendiente_Agenda_Exito=Canal01_Entregado_Pendiente_Informe.loc[Filtro_Canal01_Entregado_Pendiente_Exito].reset_index(drop=True)
Canal01_Entregado_Pendiente_Sin_Agenda_Exito=Canal01_Entregado_Pendiente_Informe.loc[~Filtro_Canal01_Entregado_Pendiente_Exito].reset_index(drop=True)

#Agenda Censosud
"""------------------------------Canal 01 Entrega con agenda Cencosub-----------------------"""
Filtro_Canal01_Entregado_Pendiente_Cencosub=(Canal01_Entregado_Pendiente_Informe['Pedido'].isin(Filtro_Agenda_Cencosub_Entrega_Pendiente))   #Excluyo con ~
Canal01_Entregado_Pendiente_Agenda_Cencosub=Canal01_Entregado_Pendiente_Informe.loc[Filtro_Canal01_Entregado_Pendiente_Cencosub].reset_index(drop=True)
Canal01_Entregado_Pendiente_Sin_Agenda_Cencosub=Canal01_Entregado_Pendiente_Informe.loc[~Filtro_Canal01_Entregado_Pendiente_Cencosub].reset_index(drop=True)


In [53]:
"""------------------------------Canal 01 Entrega con y sin agenda Exito y Cencosub-----------------------"""
Filtro_Canal01_Entregado_Pendiente_Sin_Exito_Cencosub=(Canal01_Entregado_Pendiente_Informe['Pedido'].isin(Filtro_Sin_Agenda_Cencosub_Entrega_Pendiente)) 
Canal01_Entregado_Pendiente_Sin_Agenda_Exito_Cencosub=Canal01_Entregado_Pendiente_Informe.loc[~Filtro_Canal01_Entregado_Pendiente_Sin_Exito_Cencosub].reset_index(drop=True)

In [54]:

"""----------------------------------------------Creao tabla dinamica general--------------------------------------------------------------------------------"""
Tabla_dinamica_Canal01_Entregado_Pendiente_Informe=Canal01_Entregado_Pendiente_Informe.pivot_table(index=['Pedido','Clase Pedido'],aggfunc={'Material':'count','Cantidad':sum,'Valor Neto':sum}).reset_index()

In [55]:
# Tabla dinácamica Exito
Table_dinamica_Exito_entrega_pendiente=Tabla_dinamica_Canal01_Entregado_Pendiente_Informe[Tabla_dinamica_Canal01_Entregado_Pendiente_Informe['Pedido'].isin(Filtro_Agenda_Exito_Entrega_Pendiente)]
# Tabla dinácamica Exito Cencosud
Table_dinamica_Cencosub_entrega_pendiente=Tabla_dinamica_Canal01_Entregado_Pendiente_Informe[Tabla_dinamica_Canal01_Entregado_Pendiente_Informe['Pedido'].isin(Filtro_Agenda_Cencosub_Entrega_Pendiente)]
# Tabla dinácamica sin agenda Exito y Cencosud
Table_dinamica_Sin_Exito_Cencosub_entrega_pendiente=Tabla_dinamica_Canal01_Entregado_Pendiente_Informe[~Tabla_dinamica_Canal01_Entregado_Pendiente_Informe['Pedido'].isin(Filtro_Sin_Agenda_Cencosub_Entrega_Pendiente)]


In [56]:
pedidos_Exito_Cencosub=pd.concat([Table_dinamica_Exito_entrega_pendiente['Pedido'],Table_dinamica_Cencosub_entrega_pendiente['Pedido']])
Name_ZSD037="Pedidos pendientes Exito y Cencosub"
Ruta_ZSD037_Exito_Cencosub="Archivos_CSV_Martes\\"

In [57]:
# Utilizo la transaccion para buscar ZSD037 en entrega para las fechas
url_ZSD037_Pedidos_Exito_Cencosub="Archivos_CSV\\"+str(Name_ZSD037)+".txt"

Data_ZSD037_Pedidos_Exito_Cencosub=pd.read_csv(url_ZSD037_Pedidos_Exito_Cencosub,skiprows=5,delimiter="\t")
Data_ZSD037_Pedidos_Exito_Cencosub=Funtions.Clean_Columns(Data_ZSD037_Pedidos_Exito_Cencosub)
Data_ZSD037_Pedidos_Exito_Cencosub=Funtions.default_column(default_column_Pedidos,Data_ZSD037_Pedidos_Exito_Cencosub)
Data_ZSD037_Pedidos_Exito_Cencosub=Data_ZSD037_Pedidos_Exito_Cencosub[Data_ZSD037_Pedidos_Exito_Cencosub['Nº pedido cliente'].notnull()]


### Estandarización de fechas

In [58]:
# Puede que se represente inconsistencia entre str y int por lo cual estandarizo pedidos
try:
    Data_ZSD037_Pedidos_Exito_Cencosub["Nº pedido cliente"]=Funtions.Clean_int_to_str(Data_ZSD037_Pedidos_Exito_Cencosub["Nº pedido cliente"])
    Data_ZSD037_Pedidos_Exito_Cencosub["Nº pedido cliente"]=Funtions.complete_pedidos(Data_ZSD037_Pedidos_Exito_Cencosub["Nº pedido cliente"],Agenda)
except:
    pass

In [59]:
#Dataframe de interes
Data_ZSD037_Pedidos_Pendientes_Fecha=Data_ZSD037_Pedidos_Exito_Cencosub[['Nº pedido cliente','PrimFeEntr','ÚltEntrega']].drop_duplicates()

In [60]:
#Completo Pedidos indispensable
Data_ZSD037_Pedidos_Pendientes_Fecha["Nº pedido cliente"]=Funtions.complete_pedidos(Data_ZSD037_Pedidos_Pendientes_Fecha["Nº pedido cliente"],Agenda)

In [61]:
Data_ZSD037_Pedidos_Pendientes_Fecha["Nº pedido cliente"]

0       0020679423
174     0085782046
183     0085782553
193     0085782564
203     0085782577
213     0085782604
751     0085782605
881     0085782606
891     0085782607
950     0085782642
1344    0085782643
1364    0085782644
1453    0085782645
1483    0085782646
1603    0085782650
1623    0085782657
2040    0085782658
2050    0085782660
2170    0085782661
2180    0085782662
2200    0085782771
2209    0149106725
2315    127-448885
2324    122-444641
2396    127-448893
2432     93-490535
2441     93-490639
2468     93-490644
2513     95-498897
2574     95-498907
Name: Nº pedido cliente, dtype: object

In [62]:
#---------------------------------Tabla dinamica Exito---------------------------------------------
Tabla_Agenda_Exito=pd.merge(Data_ZSD037_Pedidos_Pendientes_Fecha,Table_dinamica_Exito_entrega_pendiente,how="right",left_on="Nº pedido cliente",right_on="Pedido")[['Nº pedido cliente', 'PrimFeEntr', 'ÚltEntrega',
       'Clase Pedido', 'Cantidad', 'Material', 'Valor Neto']]

Agenda_name_Exito={"0085":"FUNZA","0020":"VEGAS","0146":"BARRANQUILLA","0149":"BUCARAMANGA","0050":"CALI","0138":"PEREIRA","0045":"SURTIMAYORISTAS"}

Tabla_Agenda_Exito["Plataforma"]=Tabla_Agenda_Exito["Nº pedido cliente"].apply(lambda x: Agenda_name_Exito[re.findall("("+"|".join(list(Agenda_name_Exito.keys()))+")",x)[0]] if re.findall("("+"|".join(list(Agenda_name_Exito.keys()))+")",x)!=[] else x)

In [63]:
Tabla_Agenda_Cencusub=pd.merge(Data_ZSD037_Pedidos_Pendientes_Fecha,Table_dinamica_Cencosub_entrega_pendiente,how="right",left_on="Nº pedido cliente",right_on="Pedido")[['Nº pedido cliente', 'PrimFeEntr', 'ÚltEntrega',
       'Clase Pedido', 'Cantidad', 'Material', 'Valor Neto']]

In [64]:
#Dataframe de interes

Tabla_Agenda_Cencusub=pd.merge(Data_ZSD037_Pedidos_Pendientes_Fecha,Table_dinamica_Cencosub_entrega_pendiente,how="right",left_on="Nº pedido cliente",right_on="Pedido")[['Nº pedido cliente', 'PrimFeEntr', 'ÚltEntrega',
       'Clase Pedido', 'Cantidad', 'Material', 'Valor Neto']]

Agenda_name_Cencosub={"93-":"MEDELLIN","122-":"BARRANQUILLA","127-":"BUCARAMANGA","95-":"CALI","60-":"BOGOTA"}

Tabla_Agenda_Cencusub["Plataforma"]=Tabla_Agenda_Cencusub["Nº pedido cliente"].apply(lambda x: Agenda_name_Cencosub[re.findall("\d*-",x)[0]] if re.findall("\d*-",x)[0] in list(Agenda_name_Cencosub.keys()) else None)

## Informe Pedidos Firme-Mercadeo-Total

In [65]:
# ZSD110 FIRME
url_ZSD110_FIRME="Archivos_CSV\\"+str(Name_ZSD110_FIRME)+".txt"
Data_ZSD110_FIRME=pd.read_csv(url_ZSD110_FIRME,skiprows=1,delimiter="\t")
Data_ZSD110_FIRME=Funtions.Clean_Columns(Data_ZSD110_FIRME)
Data_ZSD110_FIRME["Lineas"]=Funtions.Clean_Num_List(Data_ZSD110_FIRME["Lineas"])


# ZSD110 MERCADE0
url_ZSD110_MERCADEO="Archivos_CSV\\"+str(Name_ZSD110_MERCADEO)+".txt"
Data_ZSD110_MERCADEO=pd.read_csv(url_ZSD110_MERCADEO,skiprows=1,delimiter="\t")
Data_ZSD110_MERCADEO=Funtions.Clean_Columns(Data_ZSD110_MERCADEO)
Data_ZSD110_MERCADEO["Lineas"]=Funtions.Clean_Num_List(Data_ZSD110_MERCADEO["Lineas"])


# ZSD110 Pendiente Total
url_ZSD110_PENDIENTES="Archivos_CSV\\"+str(Name_ZSD110_TOTAL)+".txt"
Data_ZSD110_PENDIENTES=pd.read_csv(url_ZSD110_PENDIENTES,skiprows=1,delimiter="\t")
Data_ZSD110_PENDIENTES=Funtions.Clean_Columns(Data_ZSD110_PENDIENTES)
Data_ZSD110_PENDIENTES["Lineas"]=Funtions.Clean_Num_List(Data_ZSD110_PENDIENTES["Lineas"])

"""---------------------CAMBIAR INDICES------------------------------------------"""
Channels_Name={1:"MODERNO",2:"TRADICIONAL",7:"TIENDAS DE BELLEZA",8:"PROFESIONAL",12:"INSTITUCIONAL",14:"HARD DISCOUNTERS",15:"ECOMMERCE",19:"ALTERNATIVO",50:"ALMACENES INTERNOS"}

#Aplicar transformacion
Data_ZSD110_MERCADEO["Can.distr."]=Data_ZSD110_MERCADEO["Can.distr."].replace(Channels_Name)
Data_ZSD110_FIRME["Can.distr."]=Data_ZSD110_FIRME["Can.distr."].replace(Channels_Name)
Data_ZSD110_PENDIENTES["Can.distr."]=Data_ZSD110_PENDIENTES["Can.distr."].replace(Channels_Name)

In [66]:
#Estandarizo Datos
# Cambio a valores Numericos Mercadeo
Data_ZSD110_MERCADEO['Pedidos'],Data_ZSD110_MERCADEO['Unidades'],Data_ZSD110_MERCADEO['Valor Neto']=Funtions.Clean_Num_List(Data_ZSD110_MERCADEO['Pedidos'],Data_ZSD110_MERCADEO['Unidades'],Data_ZSD110_MERCADEO['Valor Neto'])
# Cambio a valores Numericos Firme
Data_ZSD110_FIRME['Pedidos'],Data_ZSD110_FIRME['Unidades'],Data_ZSD110_FIRME['Valor Neto']=Funtions.Clean_Num_List(Data_ZSD110_FIRME['Pedidos'],Data_ZSD110_FIRME['Unidades'],Data_ZSD110_FIRME['Valor Neto'])
# Cambio a valores Numericos Pendiente Total
Data_ZSD110_PENDIENTES['Pedidos'],Data_ZSD110_PENDIENTES['Unidades'],Data_ZSD110_PENDIENTES['Valor Neto']=Funtions.Clean_Num_List(Data_ZSD110_PENDIENTES['Pedidos'],Data_ZSD110_PENDIENTES['Unidades'],Data_ZSD110_PENDIENTES['Valor Neto'])

In [67]:
#Aplicar consolidado de todas las tablas Mercadeo Firme Total
Consolidado_Mercadeo_Firme_Total=pd.merge(pd.merge(Data_ZSD110_MERCADEO,Data_ZSD110_FIRME,how="right",on="Can.distr."),Data_ZSD110_PENDIENTES,how="right",on="Can.distr.")
Consolidado_Mercadeo_Firme_Total=Consolidado_Mercadeo_Firme_Total.fillna(0)
Consolidado_Mercadeo_Firme_Total['Can.distr.']=Consolidado_Mercadeo_Firme_Total['Can.distr.'].replace({0:"TOTAL"})

In [68]:
Consolidado_Mercadeo_Firme_Total
Column_Consolidado_Mercadeo_Firme_Total=['Can.distr.', 'Pedidos M', 'Lineas M', 'Unidades M', 'Valor Neto M',
       'Pedidos F', 'Lineas F', 'Unidades F', 'Valor Neto F', 'Pedidos T',
       'Lineas T', 'Unidades T', 'Valor Neto T']

Consolidado_Clientes=Funtions.default_column(Column_Consolidado_Mercadeo_Firme_Total,Consolidado_Mercadeo_Firme_Total)

In [69]:
#Creo tabla total de la linea de clientes

Datos_lineas_clientes=pd.DataFrame([[1,"ECOMMERCE",0],[2,"Cencosud",0],[3,"Éxito Agenda",0],
              [4,"Canal 1",0],[5,"Canales 2,7,8",0],[6,"BAZAR",0],[7,"Mercadeo",0],
              [8,"Canales 14, 19 Citas",0],[9,"Extra agenda",0]],columns=["Prioridad","Clientes","Cantidad"])

In [70]:
valor=Consolidado_Clientes[Consolidado_Clientes["Can.distr."]=='ECOMMERCE']['Lineas F'].values[0] if 'ECOMMERCE' in Consolidado_Clientes["Can.distr."].to_list() else 0



In [71]:
Columns_Consolidado_Clientes=['Pedidos M', 'Lineas M', 'Unidades M', 'Valor Neto M',
       'Pedidos F', 'Lineas F', 'Unidades F', 'Valor Neto F', 'Pedidos T',
       'Lineas T', 'Unidades T', 'Valor Neto T']
Total_Consolidado_Clientes=Consolidado_Clientes[Consolidado_Clientes['Can.distr.']=='TOTAL']
Consolidado_Clientes=Consolidado_Clientes[Consolidado_Clientes['Can.distr.']!='TOTAL']

## Funciones para Dashboard

In [72]:
def Total_Consolidado(Consolidado_Clientes,Columns_Consolidado_Clientes,Total_Consolidado_Clientes):
    Diccionario_Total_Consolidado_Clientes=Total_Consolidado_Clientes.to_dict('records')[0]
    for i in Columns_Consolidado_Clientes:
        Diccionario_Total_Consolidado_Clientes[i]=Consolidado_Clientes[i].sum()
    Consolidado_Clientes=Consolidado_Clientes.to_dict("records")
    Consolidado_Clientes.append(Diccionario_Total_Consolidado_Clientes)
    return(Consolidado_Clientes)


In [73]:
def Create_Table(Table,Name_id,fontSize='11px'):
    """
    - Table: Dataframe referencia a tabla
    - Name_id: Id de la tabla
    - Alias_libreria: Libreria Dash dash_table as dt
    """ 
    Tabla=dt.DataTable(
            id=Name_id,
            columns=[{"name": i, "id": i} for i in Table.columns],
            data=Table.to_dict("records"),
            style_data={
                'fontSize':fontSize
            },
            style_table={
                'margin': '0 auto',
                'border': '1px solid black',
                'backgroundColor': 'rgb(50, 50, 50)',
                'borderCollapse': 'collapse'
            },
            style_header={
                'fontSize':fontSize,
                'backgroundColor': 'rgb(50, 50, 50)',
                'fontWeight': 'bold',
                'border': '1px black',
                'color':'white'
            },
            style_cell={
                'textAlign': 'center',
                'border': '1px black',
                'padding': '1px',
                'width': '10px'
            },
            )
    return(Tabla)

def Create_Dropdown(Column_Table,Estados,Name_id):
        Dropdown=dcc.Dropdown(
                        id=Name_id,
                        options=[{"label": st, "value": st} for st in Estados],
                        placeholder="-Select a State-",
                        multi=True, 
                        value=Column_Table.unique(),
                        style={
                            'width': '100%',  # Ancho del dropdown
                            'font-size': '11px',  # Tamaño de fuente
                            'color': 'black',  # Color del texto
                            'border': '0px solid black',  # Borde rojo de 2px
                            'border-radius': '0px',  # Esquinas redondeadas
                            'padding': '4px',  # Relleno interno
                            'background-color': 'secondary',  # Color de fondo
                            'font-family': 'Georgia, serif',
                        }
            )
        return(Dropdown)


def data_bars(df, column):
    n_bins = 100
    bounds = [i * (1.0 / n_bins) for i in range(n_bins + 1)]
    ranges = [
        ((df[column].sum() - df[column].min()) * i)
        for i in bounds
    ]
    styles = []
    
    for i in range(1, len(bounds)):
        min_bound = ranges[i - 1]
        max_bound = ranges[i]
        max_bound_percentage = bounds[i] * 100
        styles.append({
            'if': {
                'filter_query': (
                    '{{{column}}} >= {min_bound}' +
                    (' && {{{column}}} < {max_bound}' if (i < len(bounds) - 1) else '')
                ).format(column=column, min_bound=min_bound, max_bound=max_bound),
                'column_id': column
            },
            'background': (
                """
                    linear-gradient(90deg,
                    #2FCF7F 0%,
                    #2FCF7F {max_bound_percentage}%,
                    white {max_bound_percentage}%,
                    white 100%)
                """.format(max_bound_percentage=max_bound_percentage)
            ),
            'paddingBottom': 2,
            'paddingTop': 2
        })

    return styles

def Create_Table_Condition(df,Name_id,columns,fontSize='11px'):
    table=  dt.DataTable(id=Name_id,
            data=df.to_dict('records'),
            sort_action='native',
            columns=[{'name': i, 'id': i} for i in df.columns],
            style_data_conditional=(
                data_bars(df,columns)
            ),
            style_data={
                'fontSize':fontSize
            },
            style_table={
                'margin': '0 auto',
                'border': '1px solid black',
                'backgroundColor': 'rgb(50, 50, 50)',
                'borderCollapse': 'collapse'
            },
            style_header={
                'fontSize':fontSize,
                'backgroundColor': 'rgb(50, 50, 50)',
                'fontWeight': 'bold',
                'border': '1px black',
                'color':'white'
            },
            style_cell={
                'textAlign': 'center',
                'border': '1px black',
                'padding': '1px',
                'width': '8px'
            },
            page_size=20
        )
    return(table)

In [74]:
def Create_figure(Dataframe,Column_X,list_column_y):
    Channels = Dataframe[Column_X].unique().tolist()

    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=Channels,
        y=Dataframe[list_column_y[0]].tolist(),
        name='Pedidos',
        marker_color='indianred'
    ))
        # Establecer el diseño del gráfico
    fig.add_trace(go.Bar(
        x=Channels,
        y=Dataframe[list_column_y[1]].tolist(),
        name='Lineas',
        marker_color='lightsalmon'
    ))
    fig.add_trace(go.Bar(
        x=Channels,
        y=Dataframe[list_column_y[2]].tolist(),
        name='Unidades',
        marker_color='blue'
    ))
    # Here we modify the tickangle of the xaxis, resulting in rotated labels.
    fig.update_layout(barmode='group', xaxis_tickangle=-45,xaxis=dict(tickfont=dict(size=9)),
                      legend=dict(font=dict(size=9)),margin=dict(l=0, r=0, t=10, b=0))
    return(fig)

## Preparacion de datos de entrada del dashboard

In [75]:
Lineas_table_dinamica_Exito_entrega=sum(Tabla_Agenda_Exito['Material'])+sum(Tabla_Agenda_Cencusub['Material'])

Lineas_table_dinamica_Cencosub_entrega=sum(Tabla_Agenda_Exito['Material'])+sum(Tabla_Agenda_Cencusub['Material'])

data=np.array([["Exito",2,3,Lineas_table_dinamica_Exito_entrega],["Cencosub",4,5,Lineas_table_dinamica_Cencosub_entrega]])


#Creo tabla de consolidado de lineas Exito y Cencosub

consolidado_Exito_Cencosub=pd.DataFrame(data,index=["Exito","Cencosub"],columns=["Cliente","ZVMI","FIRME","Total"])
consolidado_Exito_Cencosub["Total"]=consolidado_Exito_Cencosub["Total"].astype(float)


states_clase_pedido_Exito = Tabla_Agenda_Exito['Clase Pedido'].unique().tolist()
states_clase_pedido_Cencosub  =Tabla_Agenda_Cencusub['Clase Pedido'].unique().tolist()
states_clase_plataforma_Exito = Tabla_Agenda_Exito['Plataforma'].unique().tolist()
states_clase_plataforma_Cencosub=Tabla_Agenda_Cencusub['Plataforma'].unique().tolist()
states_Candistr=Consolidado_Clientes['Can.distr.'].unique().tolist()
        

## Contenedor 1

In [76]:
Content_1 = html.Div(children=[
    html.Div(className="row justify-content mb-3", children=[
        html.Div(className="col-md-3 "),
        html.Div(className="col-md-6 w-40", children=[
            html.Div(className="blockquote text-center", children=[
                html.H5("Consolidado"),
            ]),
            Create_Table_Condition(consolidado_Exito_Cencosub,"Consolidado_Agenda_Exito_Cencosub",'Total'),
        ]),
        html.Div(className="col-md-3 "),
    ]),
    html.Div(className="row justify-content", children=[
        html.Div(className="col-md-6 w-30 mx-auto mb-2", children=[
            html.Div(className="container bg-secondary rounded-3 border border border-success p-1 border-opacity-50 mb-2", children=[
                html.Div(className="row justify-content", children=[
                    html.Div(className="blockquote text-center text-white", children=[
                            html.H5("Agenda EXITO"),
                    ]),
                    html.Div(className="col-md-6 w-50 mx-auto", children=[
                        html.Label(['Clase Pedido'], style={'font-weight': 'bold', "text-align": "center"}),
                        Create_Dropdown(Tabla_Agenda_Exito['Clase Pedido'],states_clase_pedido_Exito,"filter_dropdown_Clase_Pedido_E"),  #Creamos Dropsown
                    ]),
                    html.Div(className="col-md-6 w-50 mx-auto", children=[
                        html.Label(['Plataforma'], style={'font-weight': 'bold', "text-align": "center"}),
                        Create_Dropdown(Tabla_Agenda_Exito['Plataforma'],states_clase_plataforma_Exito,"filter_dropdown_Plataforma_E"),   #Creamos Dropdown
                    ]),
                ]),                                   
            ]),
            Create_Table(Tabla_Agenda_Exito,"Tabla_Agenda_Exito"),    #Crear Tabla_Agenda_Exito   
        ]), 
        html.Div(className="col-md-6 w-50 mx-auto mb-2", children=[
            html.Div(className="container bg-secondary rounded-3 border border-success p-1 border-opacity-50 mb-2", children=[   
                html.Div(className="row justify-content", children=[  
                    html.Div(className="blockquote text-center text-white", children=[
                        html.H5("Agenda CENCOSUD"),
                    ]),
                    html.Div(className="col-md-6 w-50 mx-auto", children=[
                        html.Label(['Clase Pedido'], style={'font-weight': 'bold', "text-align": "start"}),
                        Create_Dropdown(Tabla_Agenda_Cencusub['Clase Pedido'],states_clase_pedido_Cencosub,"filter_dropdown_Clase_Pedido_CEN"),
                    ]),
                    html.Div(className="col-md-6 w-50 mx-auto", children=[
                        html.Label(['Plataforma'], style={'font-weight': 'bold', "text-align": "center"}),
                        Create_Dropdown(Tabla_Agenda_Cencusub['Plataforma'],states_clase_plataforma_Cencosub,"filter_dropdown_Plataforma_CEN"),   #Creamos Dropdown
                    ]),
                ]),
            ]),
            Create_Table(Tabla_Agenda_Cencusub,"Tabla entrega CENCOSUB"),   
        ]),
    ]),
])

## Contenedor 2

In [77]:
Content_2 = html.Div(children=[
    html.Div(className="row justify-content", children=[
        html.Div(className="col-md-8 w-30 p-2", children=[
                html.Div(className="blockquote text-center text-black mb-2 pe-5", children=[
                    html.H2("Total Pedidos"),
                    html.Label(['Canal de distribución'], style={'font-weight': 'bold', "text-align": "center"}),
                    Create_Dropdown(Consolidado_Clientes['Can.distr.'],states_Candistr,"filter_dropdown_Candistr"), 
                html.Div(className="row justify-content", children=[
                    Create_Table(Consolidado_Clientes,"Consolidado_Clientes","9px"),    #Crear Tabla_Agenda_Exito  
                   ]),
                html.Div(className="row justify-content mt-3", children=[
                    Create_Table(Datos_lineas_clientes,"Datos_lineas_clientes","9px"),    #Crear Tabla_Agenda_Exito 
                ]),
            ]),
        ]),
        html.Div(className="col-md-8 w-30 p-2", children=[
        ]),
        html.Div(className="col-md-4", children=[
            html.Div(className="row justify-content mx-auto p-1 mt-2",children=[
                html.Div(className="container border border-secondary border-opacity-50",style={"background-color": "#F1F1F7"},children=[
                    html.Div("Mercadeo")
                ]),
                html.Div(className="row justify-content mx-auto p-1", children=[
                    dcc.Graph(id='Grafico_Mercadeo',style={'width': '300px', 'height': '300px'},figure=Create_figure(Consolidado_Clientes,'Can.distr.',['Pedidos M','Lineas M','Unidades M'])),
                ]),
            ]),
            html.Div(className="row justify-content mx-auto p-1 mt-2",children=[
                html.Div(className="container border border-secondary border-opacity-50",style={"background-color": "#F1F1F7"},children=[
                    html.Div("Firme")
                ]),
                html.Div(className="row justify-content mx-auto p-1", children=[
                    dcc.Graph(id='Grafico_Firme',style={'width': '300px', 'height': '300px'},figure=Create_figure(Consolidado_Clientes,'Can.distr.',['Pedidos F','Lineas F','Unidades F'])),
                ]),
            ]),
            html.Div(className="row justify-content mx-auto p-1 mt-2",children=[
                html.Div(className="container border border-secondary border-opacity-50",style={"background-color": "#F1F1F7"},children=[
                    html.Div("Total")
                ]),
                html.Div(className="row justify-content mx-auto p-1", children=[
                dcc.Graph(id='Grafico_Total',style={'width': '300px', 'height': '300px'},figure=Create_figure(Consolidado_Clientes,'Can.distr.',['Pedidos T','Lineas T','Unidades T'])),
                ]),
            ]),
        ]),
    ]),
])


In [78]:
Content_2 = html.Div(children=[
        html.Div(className="row justify-content mx-auto", children=[
            html.Div(className="col-md-4",children=[
                html.Div(className="container border border-secondary border-opacity-50",style={"background-color": "#F1F1F7"},children=[
                    html.Div("Mercadeo")
                ]),
                html.Div(className="row justify-content border border-secondary border-opacity-50 mx-auto p-1", children=[
                    dcc.Graph(id='Grafico_Mercadeo',style={'width': '450px', 'height': '250px'},figure=Create_figure(Consolidado_Clientes,'Can.distr.',['Pedidos M','Lineas M','Unidades M'])),
                ]),
            ]),
            html.Div(className="col-md-4",children=[
                html.Div(className="container border border-secondary border-opacity-50",style={"background-color": "#F1F1F7"},children=[
                    html.Div("Firme")
                ]),
                html.Div(className="row justify-content border border-secondary border-opacity-50 mx-auto p-1", children=[
                    dcc.Graph(id='Grafico_Firme',style={'width': '450px', 'height': '250px'},figure=Create_figure(Consolidado_Clientes,'Can.distr.',['Pedidos F','Lineas F','Unidades F'])),
                ]),
            ]),
            html.Div(className="col-md-4",children=[
                html.Div(className="container border border-secondary border-opacity-50",style={"background-color": "#F1F1F7"},children=[
                    html.Div("Total")
                ]),
                html.Div(className="row justify-content border border-secondary border-opacity-50 mx-auto p-1", children=[
                dcc.Graph(id='Grafico_Total',style={'width': '450px', 'height': '250px'},figure=Create_figure(Consolidado_Clientes,'Can.distr.',['Pedidos T','Lineas T','Unidades T'])),
                ]),
            ]),
        ]),
    html.Div(className="row justify-content", children=[
        html.Div(className="col-md-8 w-30 p-2", children=[
            html.Div(className="row justify-content", children=[
                Create_Dropdown(Consolidado_Clientes['Can.distr.'],states_Candistr,"filter_dropdown_Candistr"), 
                Create_Table(Consolidado_Clientes,"Consolidado_Clientes","9px"),    #Crear Tabla_Agenda_Exito  
            ]),
        ]),
        html.Div(className="col-md-4 w-30 p-2", children=[
            html.Div(className="row justify-content mt-3", children=[
                Create_Table(Datos_lineas_clientes,"Datos_lineas_clientes","9px"),    #Crear Tabla_Agenda_Exito 
            ]),
        ]),
    ]),
])

## Dashboard

In [82]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = html.Div([
    dbc.Navbar(
        children=[
            dbc.Nav([  # Aquí puedes agregar elementos de la barra de navegación
                dbc.NavItem(dbc.NavLink("Acerca de", href="#")),
                dbc.NavItem(dbc.NavLink("Contacto", href="#")),
            ], className="ml-auto mb-3", navbar=True),
        ],
        color="dark",
        dark=True,
    ),
    html.Div(className="row justify-content mx-auto p-3 mt-2",children=[
        html.Main(className="main container border border-secondary border-opacity-50",style={"background-color": "#F1F1F7"},children=[
            html.Section(className="page--header",children=[
                html.Div(className="row",children=[
                    html.Div(className="col-lg-8 mx-auto",children=[
                        dbc.Nav(children=[  # Aquí puedes agregar elementos de la barra de navegación
                            html.Div(className="d-grid gap-1 col-6 mx-auto mt-4",children=[
                                dbc.Button("Linea Pedidos", className="d-grid gap-2",size='lg',id="navbar-toggler-3", color="#F1F1F7"),
                            ]),
                        ]),
                    ]),
                    html.Div(className="col-lg-2",children=[
                        html.Div(className="summary--widget",children=[
                            html.Div(className="summary--item",children=[
                                html.P("Exito",className="summary--chart",style={"color": "#009378",'width':"5","type":"bar"}),
                                html.P("2,527,371",className="summary--stats text-orange")
                            ]), 
                        ]),   
                    ]),
                    html.Div(className="col-lg-2",children=[
                        html.Div(className="summary--widget",children=[
                            html.Div(className="summary--item",children=[
                                html.P("Cencosud",className="summary--chart",style={"color": "#009378",'width':"5","type":"bar"}),
                                html.P("2,527,371",className="summary--stats text-orange")
                            ]), 
                        ]),   
                    ]),
                    dbc.Collapse(id="navbar-collapse-3",navbar=True,children=[
                        html.Div(className="row justify-content bg-white",children=[
                            Content_1
                        ]),
                    ]), 
                ]),
            ]),
        ]),
    ]),
    html.Div(className="row justify-content mx-auto p-3 mt-2",children=[
        html.Main(className="main container border border-secondary border-opacity-50",style={"background-color": "#F1F1F7"},children=[
            html.Section(className="page--header",children=[
                html.Div(className="row",children=[
                    html.Div(className="col-lg-8 mx-auto",children=[
                        dbc.Nav(children=[  # Aquí puedes agregar elementos de la barra de navegación
                            html.Div(className="d-grid gap-1 col-6 mx-auto mt-4",children=[
                                dbc.Button("Linea Pedidos", className="d-grid gap-2",size='lg',id="navbar-toggler", color="#F1F1F7"),
                            ]),
                        ]),
                    ]),
                    html.Div(className="col-lg-4 mx-auto",children=[
                        dbc.Nav(children=[  # Aquí puedes agregar elementos de la barra de navegación
                            html.Div(className="d-grid gap-1 col-6 mx-auto mt-4",children=[
                                dbc.Button("Linea Pedidos", className="d-grid gap-2",size='lg',id="navbar-toggler-8", color="#F1F1F7"),
                            ]),
                        ]),
                    ]),
                ]),
            ]),
        ]),        
        dbc.Collapse(id="navbar-collapse",navbar=True,children=[
            html.Div(className="row justify-content bg-white",children=[
                Content_2
            ]),
        ]),   
    ]),
    html.Section(className="row justify-content bg-warning",children=[
        dbc.Nav(children=[  # Aquí puedes agregar elementos de la barra de navegación
            dbc.Button("Linea Pedidos", id="navbar-toggler-2", color="secondary"),
        ]),
        dbc.Collapse(
            dbc.Nav(className="ml-auto", navbar=True,children=[  # Aquí puedes agregar elementos de la barra de navegación
                html.Div(className="container bg-secundary",children=[
                    dcc.Tabs(id="tabs-example-graph-2", value='tab-1', children=[
                        dcc.Tab(label='Analisis de requisitos', value='tab-1'),
                    ]),
                ])
            ]),
            id="navbar-collapse-2",
            navbar=True,
        ),    
    ]),
])

@app.callback(
    Output("Tabla_Agenda_Exito", "data"),
    Output("Tabla entrega CENCOSUB", "data"), 
    Output("Consolidado_Agenda_Exito_Cencosub", "data"),
    Output("navbar-collapse", "is_open"),
    Input("filter_dropdown_Clase_Pedido_E", "value"),
    Input("filter_dropdown_Plataforma_E", "value"),
    Input("filter_dropdown_Clase_Pedido_CEN", "value"),
    Input("filter_dropdown_Plataforma_CEN", "value"),
    Input("navbar-toggler", "n_clicks"),
    State("navbar-collapse", "is_open")
)
def toggle_navbar_collapse(State_Clase_Pedido_EXT,State_Plataforma_EXT,State_Clase_Pedido_CEN,State_Plataforma_CEN,n_clicks, is_open):
    """
    -State_Clase_Pedido_EXT: Input del callback Input("filter_dropdown_Clase_Pedido_E", "value") -> List()
    -State_Plataforma_EXT: Input del callback Input("filter_dropdown_Plataforma_E", "value") -> List()
    -State_Clase_Pedido_CEN: Input del callback Input("filter_dropdown_Clase_Pedido_CEN", "value") -> List()
    -State_Plataforma_CEN: Input del callback Input("filter_dropdown_Plataforma_CEN", "value") -> List()
    -n_clicks: Input del callback Input("navbar-toggler", "n_clicks")
    -s_open: State del callback State("navbar-collapse", "is_open")
    ------
    Return(Output_Exito.to_dict("records")->dict,Output_Cencosub.to_dict("records")->dict,dff3.to_dict("records")->dict,is_open or not is_open->bool)
    """
    # Filtro para agenda Exito y Cencosud con columna 'Clase Pedido' y 'Plataforma'
    Output_Exito = Tabla_Agenda_Exito[Tabla_Agenda_Exito['Clase Pedido'].isin(State_Clase_Pedido_EXT) | Tabla_Agenda_Exito['Plataforma'].isin(State_Plataforma_EXT)]
    Output_Cencosub = Tabla_Agenda_Cencusub[Tabla_Agenda_Cencusub['Clase Pedido'].isin(State_Clase_Pedido_CEN)| Tabla_Agenda_Cencusub['Plataforma'].isin(State_Plataforma_CEN)]
    # Filtro para Clase pedido ZVMI de agenda Exito y Cencosud
    Output_Exito_ZVMI=Output_Exito[Output_Exito['Clase Pedido']=='ZVMI']
    Output_Cencosub_ZVMI=Output_Cencosub[Output_Cencosub['Clase Pedido']=='ZVMI']

    # Tabla consolidado
    Output_Exito_ZED1_ZEDI=Output_Exito[(Output_Exito['Clase Pedido']=='ZED1') | (Output_Exito['Clase Pedido']=='ZEDI')] 
    Output_Cencosub_ZED1_ZEDI=Output_Cencosub[(Output_Cencosub['Clase Pedido']=='ZED1') | (Output_Cencosub['Clase Pedido']=='ZEDI')]
    # Contabilizo total de las lineas de las agenda Exito y Cencosub Total ZVMI, Total Firme y Total
    consolidado_Exito_Cencosub["ZVMI"][0]=Output_Exito_ZVMI["Material"].sum()
    consolidado_Exito_Cencosub["ZVMI"][1]=Output_Cencosub_ZVMI["Material"].sum()
    consolidado_Exito_Cencosub["FIRME"][0]=Output_Exito_ZED1_ZEDI["Material"].sum()
    consolidado_Exito_Cencosub["FIRME"][1]=Output_Cencosub_ZED1_ZEDI["Material"].sum()
    consolidado_Exito_Cencosub["Total"]=consolidado_Exito_Cencosub["ZVMI"]+consolidado_Exito_Cencosub["FIRME"]
    output_consolidado_Exito_Cencosub=consolidado_Exito_Cencosub
    if n_clicks:
        return Output_Exito.to_dict("records"),Output_Cencosub.to_dict("records"),output_consolidado_Exito_Cencosub.to_dict("records"),not is_open
    return Output_Exito.to_dict("records"),Output_Cencosub.to_dict("records"),output_consolidado_Exito_Cencosub.to_dict("records"),is_open

@app.callback(
    Output("Consolidado_Clientes", "data"),
    Output("Grafico_Mercadeo", "figure"),
    Output('Grafico_Firme', "figure"),  
    Output('Grafico_Total', "figure"),
    Output("navbar-collapse-2", "is_open"),
    Input("filter_dropdown_Candistr", "value"),
    Input("navbar-toggler-2", "n_clicks"),
    State("navbar-collapse-2", "is_open"),
)
def toggle_navbar_collapse(State_Candistr,n_clicks_2, is_open_2):
    Output_Exito_ZVMI=Tabla_Agenda_Exito[Tabla_Agenda_Exito['Clase Pedido']=='ZVMI']
    Output_Cencosub_ZVMI=Tabla_Agenda_Cencusub[Tabla_Agenda_Cencusub['Clase Pedido']=='ZVMI']
    Output_Exito_Firme=Tabla_Agenda_Exito[Tabla_Agenda_Exito['Clase Pedido']!='ZVMI']
    Output_Consolidado=Consolidado_Clientes[Consolidado_Clientes['Can.distr.'].isin(State_Candistr)]
    Output_Consolidado_figure=Consolidado_Clientes[Consolidado_Clientes['Can.distr.'].isin(State_Candistr)]
    figure_mercadeo=Create_figure(Output_Consolidado_figure,'Can.distr.',['Pedidos M','Lineas M','Unidades M'])
    figure_firme=Create_figure(Output_Consolidado_figure,'Can.distr.',['Pedidos F','Lineas F','Unidades F'])
    figure_total=Create_figure(Output_Consolidado_figure,'Can.distr.',['Pedidos T','Lineas T','Unidades T'])
    Output_Consolidado=Total_Consolidado(Output_Consolidado,Columns_Consolidado_Clientes,Total_Consolidado_Clientes)
    if n_clicks_2:
        return Output_Consolidado,figure_mercadeo,figure_firme,figure_total,not is_open_2
    return Output_Consolidado,figure_mercadeo,figure_firme,figure_total,is_open_2

@app.callback(
    Output("navbar-collapse-3", "is_open"),
    Input("navbar-toggler-3", "n_clicks"),
    State("navbar-collapse-3", "is_open"),
)
def toggle_navbar_collapse(n_clicks_3, is_open_3):
    if n_clicks_3:
        return not is_open_3
    return is_open_3


 #<ul class="breadcrumb"> <li class="breadcrumb-item active"><span>Dashboard</span></li></ul> </div><div class="col-lg-6"> <div class="summary--widget"> <div class="summary--item"> <p class="summary--chart" data-trigger="sparkline" data-type="bar" data-width="5" data-height="38" data-color="#009378"><canvas width="71" height="38" style="display: inline-block; width: 71px; height: 38px; vertical-align: top;"></canvas></p><p class="summary--title">This Month</p><p class="summary--stats text-green">2,371,527</p></div><div class="summary--item"> <p class="summary--chart" data-trigger="sparkline" data-type="bar" data-width="5" data-height="38" data-color="#e16123"><canvas width="71" height="38" style="display: inline-block; width: 71px; height: 38px; vertical-align: top;"></canvas></p><p class="summary--title">Last Month</p><p class="summary--stats text-orange">2,527,371</p></div></div></div></div></div></section>

app.run(host='0.0.0.0', port=8000, debug=False)

Dash is running on http://0.0.0.0:8000/

Dash is running on http://0.0.0.0:8000/

Dash is running on http://0.0.0.0:8000/

Dash is running on http://0.0.0.0:8000/

Dash is running on http://0.0.0.0:8000/

Dash is running on http://0.0.0.0:8000/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.2.101:8000/ (Press CTRL+C to quit)


192.168.2.101 - - [30/Jul/2023 23:44:26] "GET / HTTP/1.1" 200 -
192.168.2.101 - - [30/Jul/2023 23:44:28] "GET /_dash-layout HTTP/1.1" 200 -
192.168.2.101 - - [30/Jul/2023 23:44:28] "GET /_dash-dependencies HTTP/1.1" 200 -
192.168.2.101 - - [30/Jul/2023 23:44:31] "GET /_favicon.ico?v=2.7.0 HTTP/1.1" 200 -
192.168.2.101 - - [30/Jul/2023 23:44:31] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
192.168.2.101 - - [30/Jul/2023 23:44:31] "GET /_dash-component-suites/dash/dash_table/async-highlight.js HTTP/1.1" 200 -
192.168.2.101 - - [30/Jul/2023 23:44:31] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
192.168.2.101 - - [30/Jul/2023 23:44:32] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
192.168.2.101 - - [30/Jul/2023 23:44:32] "GET /_dash-component-suites/dash/dash_table/async-table.js HTTP/1.1" 200 -
192.168.2.101 - - [30/Jul/2023 23:44:32] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.2.101 - - [30/Jul/2023 23

In [ ]:
    elif tab == 'tab-2-example-graph':
        Output_Exito_ZVMI=Tabla_Agenda_Exito[Tabla_Agenda_Exito['Clase Pedido']=='ZVMI']
        Output_Cencosub_ZVMI=Tabla_Agenda_Cencusub[Tabla_Agenda_Cencusub['Clase Pedido']=='ZVMI']
        Output_Exito_Firme=Tabla_Agenda_Exito[Tabla_Agenda_Exito['Clase Pedido']!='ZVMI']
        Output_Consolidado=Consolidado_Clientes[Consolidado_Clientes['Can.distr.'].isin(State_Candistr)]
        Output_Consolidado_figure=Consolidado_Clientes[Consolidado_Clientes['Can.distr.'].isin(State_Candistr)]
        figure_mercadeo=Create_figure(Output_Consolidado_figure,'Can.distr.',['Pedidos M','Lineas M','Unidades M'])
        figure_firme=Create_figure(Output_Consolidado_figure,'Can.distr.',['Pedidos F','Lineas F','Unidades F'])
        figure_total=Create_figure(Output_Consolidado_figure,'Can.distr.',['Pedidos T','Lineas T','Unidades T'])
        Output_Consolidado=Total_Consolidado(Output_Consolidado,Columns_Consolidado_Clientes,Total_Consolidado_Clientes)
        Lineas_Output_Exito_ZVMI=Output_Exito_ZVMI['Material'].sum()
        Lineas_Output_Cencosub_ZVMI=Output_Cencosub_ZVMI['Material'].sum()
        Lineas_Output_Exito_Firme=Output_Exito_Firme['Material'].sum()
        Output_Datos_lineas_clientes=Datos_lineas_clientes.to_dict("records")
        # Update table Datos_lineas_clientes
        Valor_Eccomerce=Consolidado_Clientes[Consolidado_Clientes["Can.distr."]=='ECOMMERCE']['Lineas F'].values[0] if 'ECOMMERCE' in Consolidado_Clientes["Can.distr."].to_list() else 0
        Valor_Mercadeo=Consolidado_Clientes[Consolidado_Clientes["Can.distr."]=='MODERNO']['Lineas F'].values[0] if 'MODERNO' in Consolidado_Clientes["Can.distr."].to_list() else 0
        Output_Datos_lineas_clientes[0]["Cantidad"]=Valor_Eccomerce
        Output_Datos_lineas_clientes[1]["Cantidad"]=Lineas_Output_Cencosub_ZVMI
        Output_Datos_lineas_clientes[2]["Cantidad"]=Lineas_Output_Exito_ZVMI
        Output_Datos_lineas_clientes[3]["Cantidad"]=Valor_Mercadeo-Lineas_Output_Exito_Firme  #Preguntar por cencosub
        Output_Datos_lineas_clientes[4]["Cantidad"]=Consolidado_Clientes["Lineas F"].loc[1:3].sum()
        Output_Datos_lineas_clientes[5]["Cantidad"]=Consolidado_Clientes["Lineas F"].loc[4]
        Output_Datos_lineas_clientes[6]["Cantidad"]=Consolidado_Clientes["Lineas M"].sum()
        Output_Datos_lineas_clientes[7]["Cantidad"]=Consolidado_Clientes["Lineas F"].loc[5]+Consolidado_Clientes["Lineas F"].loc[7]
        Output_Datos_lineas_clientes[8]["Cantidad"]=Consolidado_Clientes["Lineas T"].loc[0]-Lineas_Output_Exito_ZVMI-Consolidado_Clientes["Lineas F"].loc[1:3].sum()
        return dash.no_update,dash.no_update,dash.no_update,Output_Consolidado,Output_Datos_lineas_clientes,figure_mercadeo,figure_firme,figure_total

: 

In [ ]:
Consolidado_Clientes

: 

# Headrer

from dash import Dash, dash_table
import pandas as pd

app = Dash(__name__)

app.layout = dash_table.DataTable(
    columns=[
        {"name": ["", "Year"], "id": "year"},
        {"name": ["City", "Montreal"], "id": "montreal"},
        {"name": ["City", "Toronto"], "id": "toronto"},
        {"name": ["City", "Ottawa"], "id": "ottawa"},
        {"name": ["City", "Vancouver"], "id": "vancouver"},
        {"name": ["Climate", "Temperature"], "id": "temp"},
        {"name": ["Climate", "Humidity"], "id": "humidity"},
    ],
    data=[
        {
            "year": i,
            "montreal": i * 10,
            "toronto": i * 100,
            "ottawa": i * -1,
            "vancouver": i * -10,
            "temp": i * -100,
            "humidity": i * 5,
        }
        for i in range(10)
    ],
    merge_duplicate_headers=True,
)

if __name__ == '__main__':
    app.run(debug=True)


In [ ]:
import dash_bootstrap_components as dbc
from dash import Input, Output, State, html

collapse = html.Div(
    [
        dbc.Button(
            "Open collapse",
            id="horizontal-collapse-button",
            className="mb-3",
            color="primary",
            n_clicks=0,
        ),
        html.Div(
            dbc.Collapse(
                dbc.Card(
                    dbc.CardBody(
                        "This content appeared horizontally due to the "
                        "`dimension` attribute"
                    ),
                    style={"width": "400px"},
                ),
                id="horizontal-collapse",
                is_open=False,
                dimension="width",
            ),
            style={"minHeight": "100px"},
        ),
    ]
)


@app.callback(
    Output("horizontal-collapse", "is_open"),
    [Input("horizontal-collapse-button", "n_clicks")],
    [State("horizontal-collapse", "is_open")],
)
def toggle_collapse(n, is_open):
    if n:
        return not is_open
    return is_open

: 

In [ ]:
consolidado_Exito_Cencosub

: 